In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pip install -q git+https://github.com/eaedk/testing-zindi-package.git

In [ ]:
# Import Zindian class from the package
from zindi.user import Zindian
# create a user object
my_username = "Johannes"
user = Zindian(username = my_username)

In [ ]:
user.select_a_challenge()                               # Select a Zindi challeng

In [ ]:
user.download_dataset(destination="dataset") # Download the dataset of the selected challenge

In [ ]:
!pip install -qU 'git+https://github.com/PyTorchLightning/lightning-flash.git#egg=lightning-flash[image]'

In [ ]:
train = pd.read_csv('./dataset/Train.csv')
test = pd.read_csv('./dataset/Test.csv')
ss = pd.read_csv('./dataset/SampleSubmission.csv')

In [ ]:
!unzip ./dataset/coral_image.zip

In [ ]:
train.label.value_counts()

In [ ]:
import glob

total_path = glob.glob('./coral_image/*.*')
# total_path

In [ ]:
list_train = pd.DataFrame()
list_train['image_path'] = total_path
list_train['Image_ID'] = list_train['image_path'].apply(lambda x: x.split('/')[2])
list_train['Image_ID'] = list_train['Image_ID'].apply(lambda x: x.split(".")[0])
train = pd.merge(train, list_train, on=["Image_ID"]).drop('Image_ID', axis=1)
train.head()

In [ ]:
test.shape

In [ ]:
test = pd.merge(test, list_train, on=["Image_ID"])

In [ ]:
train = train[train['label']!='Soft Coral']
train = train[train['label']!='Other Invertebrates']
# train = train[train['label']!='Other']
train.label.value_counts()

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(train['label'])
train['label'] = le.transform(train['label'])
# train['label'] = train['label'].astype(float)

In [ ]:
train.label.value_counts()

In [ ]:
import torch
import flash
from flash.image import ImageClassificationData, ImageClassifier

In [ ]:
images = [i for i in train['image_path']]
label = [i for i in train['label']]

datamodule = ImageClassificationData.from_files(
    train_files = images,
    train_targets = label,image_size=(128, 128),val_split=0.1
)


In [ ]:
datamodule.show_train_batch()

In [ ]:
import flash
from flash.image import ImageClassifier

#  2. Build the task
# model = ImageClassifier(backbone="resnext50_32x4d", num_classes=datamodule.num_classes, multi_label=True
#                        )
model = ImageClassifier(backbone="resnet18", num_classes=datamodule.num_classes,
            
                       )

In [ ]:
trainer = flash.Trainer(max_epochs=3, gpus=torch.cuda.device_count())

In [ ]:
%%time
trainer.finetune(model, datamodule=datamodule, strategy="freeze")

In [ ]:
x = [i for i in test['image_path']]

label = model.predict(x)
test['label'] = label
ss['Image_ID'] = test['Image_ID']
ss['label'] = test['label']
ss.label.value_counts()

In [ ]:
ss['label'] = list(le.inverse_transform(ss['label']))

In [ ]:
# torch.save(model.state_dict(), 'model.pt')

In [ ]:
ss['label'] = list(le.inverse_transform(ss['label']))
ss.to_csv("./submission.csv", index=False)
user.submit(filepaths=["./submission.csv"], comments=["Other"])

In [ ]:
user.submission_board()

In [ ]:
### the gpu crushes so i split the data in batches manually
# import itertools

# def batch(iterable, n=1):
#     l = len(iterable)
#     for ndx in range(0, l, n):
#         yield iterable[ndx:min(ndx + n, l)]

# x = [i for i in test['image_path'] ]
# x1 = model.predict(list(batch(x, 2000))[0])
# x2 = model.predict(list(batch(x, 2000))[1])
# x3 = model.predict(list(batch(x, 2000))[2])
# x4 = model.predict(list(batch(x, 2000))[3])
# x5 = model.predict(list(batch(x, 2000))[4])
# x6 = model.predict(list(batch(x, 2000))[5])

# label = x1+x2+x3+x4+x5+x6

# test['label'] = label
# ss['Image_ID'] = test['Image_ID']
# ss['label'] = test['label']
# ss.label.value_counts()
